In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper')
sns.set_style('ticks')

In [2]:
## utils
def euclidean_distance(x, y):
    return np.linalg.norm(x - y)

def find_closest_cluster(points,centroids):
    min_distance = np.inf
    closest_cluster = -1
    for i,centroid in enumerate(centroids):
        distance = euclidean_distance(points,centroid)
        if distance < min_distance:
            min_distance = distance
            closest_cluster = i
    return closest_cluster

def update_centroid(data,cluster_indices):
    if len(cluster_indices) == 0:
        return None 
    return np.mean(data[cluster_indices],axis=0)

In [3]:
#Step 1: Initial Crawling
def initial_clustering(data, threshold):
  """
  Performs initial clustering by assigning points to closest clusters.
  """
  clusters = []
  centroids = []
  for i in range(len(data)):
    closest_cluster = find_closest_cluster(data[i], centroids)
    if closest_cluster == -1:
      centroids.append(data[i])
      clusters.append([i])
    else:
      distance = euclidean_distance(data[i], centroids[closest_cluster])
      if distance <= threshold:
        clusters[closest_cluster].append(i)
      else:
        centroids.append(data[i])
        clusters.append([i])
  return clusters, centroids



In [4]:
def adjust_clusters(data, clusters, centroids):
    adjustments = 0
    i = 0
    while i < len(data):
        closest_cluster = find_closest_cluster(data[i], centroids)
        if closest_cluster != i:
            old_cluster = clusters[i]
            clusters[i].remove(i)
            clusters[closest_cluster].append(i)
            adjustments += 1
            i += 1

    # Update centroids outside the loop to avoid multiple updates within the same iteration
    for cluster_idx in range(len(clusters)):
        centroids[cluster_idx] = update_centroid(data, clusters[cluster_idx])

    return clusters, centroids, adjustments

In [5]:
#Step 3: merge_clusters
def merge_clusters(data,clusters,centroids,cluster_threshold,max_loops=20):
    loop_counter = 1
    while True:
        merge_counter = 0
        for i in range(len(clusters)):
            if clusters[i] is None:
                continue
            for j in range(i+1,len(clusters)):
                if clusters[j] is None:
                    continue
                distance = euclidean_distance(centroids[i],centroids[j])
                if distance <= cluster_threshold:
                    merged_centroid = (centroids[i] + centroids[j])/2
                    keep_points_i = []
                    keep_points_j = []
                    for p_idx in clusters[i]:
                        if euclidean_distance(data[p_idx],merged_centroid) <= cluster_threshold:
                            keep_points_i.append(p_idx)
                    for p_idx in clusters[j]:
                        if euclidean_distance(data[p_idx],merged_centroid) <= cluster_threshold:
                            keep_points_j.append(p_idx)
                    if len(keep_points_i) + len(keep_points_j) == len(clusters[i]) + len(clusters[j]):
                        clusters[i] = keep_points_i
                        clusters[j] = keep_points_j
                        centroids[i] = merged_centroid
                        centroids[j] = None 
                        merge_counter += 1
        if merge_counter == 0:
            break
        loop_counter += 1
        if loop_counter > max_loops:
            cluster_threshold *=0.95
    #remove empty clusters
    clusters = [cluster for cluster in clusters if cluster is not None]
    centroids = [centroid for centroid in centroids if centroid is not None]
    return clusters,centroids

In [6]:
def dCrawler_main(data,threshold,cluster_threshold):
    clusters,centroids = initial_clustering(data,threshold)
    clusters,centroids,_ = adjust_clusters(data,clusters,centroids)
    clusters,centroids = merge_clusters(data,clusters,centroids,cluster_threshold)
    return clusters,centroids

In [18]:
data = np.random.rand(100,2) #100 points in 2D

In [ ]:
clusters,centroids = initial_clustering(data,0.2)
#plot 
plt.figure(figsize=(5,5))
for cluster in clusters:
    plt.scatter(data[cluster,0],data[cluster,1],label=f'Cluster{cluster}')
plt.scatter(np.array(centroids)[:,0],np.array(centroids)[:,1],c='b',marker='x')
#draw a circle around each centroid
for centroid in centroids:
    circle = plt.Circle(centroid, 0.05, color='b', fill=False)
    plt.gcf().gca().add_artist(circle)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Initial Clustering')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.show()

In [22]:
adj_cluster,adj_centroids,_ = adjust_clusters(data,clusters,centroids)

KeyboardInterrupt: 

In [ ]:
#plot the adjusted clusters
plt.figure(figsize=(5,5))
for cluster in adj_cluster:
    plt.scatter(data[cluster,0],data[cluster,1],label=f'Cluster{cluster}')
plt.scatter(np.array(adj_centroids)[:,0],np.array(adj_centroids)[:,1],c='b',marker='x')
#draw a circle around each centroid

In [ ]:

#Step 1: Initial Crawling
def initial_clustering(data, threshold):
  """
  Performs initial clustering by assigning points to closest clusters.
  """
  clusters = []
  centroids = []
  for i in range(len(data)):
    closest_cluster = find_closest_cluster(data[i], centroids)
    if closest_cluster == -1:
      centroids.append(data[i])
      clusters.append([i])
    else:
      distance = euclidean_distance(data[i], centroids[closest_cluster])
      if distance <= threshold:
        clusters[closest_cluster].append(i)
      else:
        centroids.append(data[i])
        clusters.append([i])
  return clusters, centroids

#Step 2: Adjust clusters
def adjust_clusters(data,clusters,centroids):
    adjustments = 0
    i=1
    while i<=len(data):
        closest_cluster = find_closest_cluster(data[i-1],centroids)
        if closest_cluster != clusters[i-1]:
            old_cluster = clusters[i-1]
            clusters[i-1].remove(i-1)
            clusters[closest_cluster].append(i-1)
            centroids[old_cluster] = update_centroid(data,clusters[old_cluster])
            centroids[closest_cluster] = update_centroid(data,clusters[closest_cluster])
            adjustments += 1
            i+=1
    return clusters,centroids,adjustments #maybe needed? counters?


#Step 3: merge_clusters
def merge_clusters(data,clusters,centroids,cluster_threshold,max_loops=20):
    loop_counter = 1
    while True:
        merge_counter = 0
        for i in range(len(clusters)):
            if clusters[i] is None:
                continue
            for j in range(i+1,len(clusters)):
                if clusters[j] is None:
                    continue
                distance = euclidean_distance(centroids[i],centroids[j])
                if distance <= cluster_threshold:
                    merged_centroid = (centroids[i] + centroids[j])/2
                    keep_points_i = []
                    keep_points_j = []
                    for p_idx in clusters[i]:
                        if euclidean_distance(data[p_idx],merged_centroid) <= cluster_threshold:
                            keep_points_i.append(p_idx)
                    for p_idx in clusters[j]:
                        if euclidean_distance(data[p_idx],merged_centroid) <= cluster_threshold:
                            keep_points_j.append(p_idx)
                    if len(keep_points_i) + len(keep_points_j) == len(clusters[i]) + len(clusters[j]):
                        clusters[i] = keep_points_i
                        clusters[j] = keep_points_j
                        centroids[i] = merged_centroid
                        centroids[j] = None 
                        merge_counter += 1
        if merge_counter == 0:
            break
        loop_counter += 1
        if loop_counter > max_loops:
            cluster_threshold *=0.95
    #remove empty clusters
    clusters = [cluster for cluster in clusters if cluster is not None]
    centroids = [centroid for centroid in centroids if centroid is not None]
    return clusters,centroids